In [1]:
import argparse
from hqq.core.quantize import *
import torch
import os
import sys
import json
sys.path.append('../saving')
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"
from transformers import AutoTokenizer, MixtralConfig 
from modeling_phimoe import PhimoeForCausalLM
# from modeling_mixtral_sparsity import MixtralForCausalLM, load_thresholds

import numpy as np

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 如果使用 GPU
    torch.backends.cudnn.deterministic = True  # 确保 CUDA 操作是确定性的
    torch.backends.cudnn.benchmark = False  # 禁用 CUDA 的自动优化


from torch.utils.data import DataLoader
from tqdm import tqdm
# from configuration_phimoe import PhiMoEConfig
# from modeling_phimoe import PhiMoEForCausalLM, load_thresholds, set_profile_mode
set_seed(42)

import transformers
from hqq.models.base import BaseHQQModel
from accelerate import init_empty_weights
from hqq.models.hf.mixtral import MixtralPatch

with open("/home/bcds/On-the-Fly_MoE_Inference/quantize/device_map.json", "r") as f:
    device_map = json.load(f)

with open('../path.json', 'r') as file:
    paths = json.load(file)
    # threshold_path = paths.get('mixtral_threshold','')
    # model_name = paths.get("mixtral","")
    threshold_path = paths.get('phi_threshold','')
    model_name = paths.get("phi","")

# filepath = str(0.5).replace('.', '_')
# load_thresholds(f'{threshold_path}/thresholds_{filepath}.pt', use_type="gate", zero=False)
# set_profile_mode(False)
# llm = get_model(model_name, device_map)
# configuration = PhiMoEConfig(sparsity_selection="gate")
llm = PhimoeForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    # config=configuration,
    torch_dtype=torch.float16,
    trust_remote_code=False,  
    # attn_implementation="flash_attention_2"
)
    
print(f'with sparsity of {0.5}')

tokenizer = AutoTokenizer.from_pretrained(model_name)

from datasets import load_dataset
tokenizer.pad_token = tokenizer.eos_token

import random
from datasets import load_dataset
valenc = []
# seqlen=512
seqlen=512
random.seed(0)
llm.eval()
llm.config.use_cache=False
valdata = load_dataset('/home/bcds/venv/dilab/floe/dataset/wikitext/wikitext-2-raw-v1', split='test')

valdata_tok = tokenizer("\n\n".join(valdata['text'][:1000]), return_tensors='pt')

/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='longrope': {'long_mscale', 'short_mscale'}
This model has set a `original_max_position_embeddings` field, to be used together with `max_position_embeddings` to determine a scaling factor. Please set the `factor` field of `rope_scaling`with this ratio instead -- we recommend the use of this field over `original_max_position_embeddings`, as it is compatible with most model architectures.
Loading checkpoint shards: 100%|██████████| 17/17 [00:22<00:00,  1.31s/it]


with sparsity of 0.5


In [2]:

class TokenizerWrapper:
    def __init__(self, input_ids):
        self.input_ids = input_ids
# valenc = TokenizerWrapper(valenc)
# print(valdata)
testenc = valdata_tok.input_ids


# Calculate number of samples
nsamples = testenc.numel() // seqlen

# List to store negative log likelihoods
nlls = []
print(f"nsamples {nsamples}")

from tqdm import trange
with torch.no_grad():
    # Loop through each batch
    for i in trange(0, nsamples, 1):
        # Calculate end index
        j = min(i+1, nsamples)

        # Prepare inputs and move to device
        inputs = testenc[:,(i * seqlen):(j * seqlen)].to(llm.device)
        inputs = inputs.reshape(j-i, seqlen)

        # Forward pass through the model
        lm_logits = llm(inputs,use_cache=False).logits

        # Shift logits and labels for next token prediction
        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = inputs[:, 1:]

        # Compute loss
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.reshape(-1, shift_logits.size(-1)), shift_labels.reshape(-1))
        # Calculate negative log likelihood
        neg_log_likelihood = loss.float() * seqlen * (j-i)
        # print(neg_log_likelihood.item())
        # Append to list of negative log likelihoods
        nlls.append(torch.tensor(neg_log_likelihood.item()))
        # print(nlls)
        # del inputs, lm_logits, shift_logits, shift_labels, loss
        torch.cuda.empty_cache()

# Compute perplexity
ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * seqlen))

# Empty CUDA cache to save memory
torch.cuda.empty_cache()

print('PPL: {:.4f}'.format(ppl.item()))


# avg_list = []
# for layerid in range(32):
#     for expertid in range(16):
#         avg_list.append(llm.model.layers[layerid].block_sparse_moe.experts[expertid].get_ratio())

# print('Average Sparsity: ', f'{sum(avg_list)/len(avg_list):.4f}')
# print('Max Sparsity: {:.4f}'.format(max(avg_list)))
# print('Min Sparsity: {:.4f}'.format(min(avg_list)))


nsamples 164


100%|██████████| 164/164 [00:27<00:00,  5.93it/s]

PPL: 5.1181
